Привет, Настя! Меня зовут Максим Слотин, я буду делать проекта. Давай будем общаться на «ты».

Ты проделала большую исследовательскую работу! Твой проект очень хорошо структурирован, ты делаешь хорошие понятные выводы. Мои комментарии будут в ячейках "Комментарий от ревьюера".

<div class="alert alert-danger"> 
<h2> Комментарии от ревьюера №1</h2>
Красным цветом обозначены комментарии, на которые тебе нужно обратить внимание в первую очередь, чтобы я принял проект. 
</div>
<div class="alert alert-warning">
<h2> Комментарии от ревьюера №1</h2>
Желтым цветом отмечено то, что ты можешь сделать немного по другому. Ты можешь учесть эти комментарии при выполнении следующих проектов.
</div>
<div class="alert alert-success">
<h2> Комментарии от ревьюера №1</h2>
Зелёным цветом обозначены твои удачные решения, которые ты без сомнения можешь применять в будущих проектах.
</div>

#### Пожалуста, не удаляй мои комментарии и относись к ним бережно. При повторной проверке, я буду ориентироваться на них. 

###### Можешь под моими комментариями оставлять свои, в которых опиши свои испарвления. Это облегчит процесс проверки и мне, и тебе  :) 

Например, вот так:

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ ОТ УЧЕНИКА:</b>
</div>
---

</div>
<div class="alert alert-success">
<h2> Комментарии от ревьюера №3</h2>
    
У тебя в итоге вышел отличный проект)
    
Спасибо, что довольно внимательно и терпеливо отреагировала на мои комментарии 😊
    
Желаю тебе дальше успехов в учебе)
</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [38]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
#data.info()
print(data.head(20))

# print(data['days_employed'].value_counts()) # есть - общий трудовой стаж в днях



#print(data[data['total_income'].isnull()].head())




    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Вывод

при первичной проверки были обнаружены пропуски в столбцах days_employed и total_income

### Шаг 2. Предобработка данных

### Обработка пропусков

In [48]:
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
    
ratio_days_employed = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-18)*365)
days_employed_mean = data['days_employed'].mean()
total_income_median = data['total_income'].median()
dob_years_mean = data['dob_years'].mean()

#data['dob_years'] = data['dob_years'].fillna(value = dob_years_mean)
data['total_income'] = data['total_income'].fillna(value=total_income_median)
data['days_employed'] = data['days_employed'].fillna(value=days_employed_mean)
data['days_employed'] = data['days_employed'].apply(negative_to_pozitive)
data['days_employed'] = data['days_employed'].fillna(value = ratio_days_employed)

data.loc[data['income_type'] == 'пенсионер', 'days_employed'] =data.loc[data['income_type'] == 'пенсионер', 'days_employed'] / 24

data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))
#print(data.sort_values(by = 'days_employed', ascending = False))

#print(data.loc[data.loc[:, 'income_type'] == 'пенсионер'] )

print(data.head(10))


   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0      24.614155         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   СРЕДНЕЕ             1   
8         2    6929.865299         35    ВЫСШЕЕ             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

1. Для работы с пропусками в количественных переменных для столбца days_employed был взят средний общий трудовой стаж в днях, для total_income медиана, потому что средние в заработной плате брать не совсем корректно, в данные могут попасть люди с большой разницей зарплаты относительно остальных людей
2. В столбце days_employed были отрицательные значения по трудовому стажу, они были заменены на положительные.


<div class="alert alert-danger">
<h2> Комментарии от ревьюера №1</h2>
    
Молодец, что обратила внимание на отрицательные значения в колонке `days_employed`, однако, там есть еще один сюрприз.
    
Посмотри на максимальные значения, которые в этой колонке. Ты обнаружишь, что есть очень большие стажи (слишком большие для человка), сразу скажу, что проблема не в долгожителях, а в том, что есть значения, которые проставлены не в днях, а в часах. Попробуй обнаружить сет данных, который представлен в часах и исправь этот момент
</div>

КОММЕНТАРИЙ ОТ УЧЕНИКА: Не совсем  понимаю как выделить данную группу, можете помочь? 

<div class="alert alert-warning">
<h2> Комментарии от ревьюера №2</h2>
    
Люди старше 50 лет имеют данные в формате часов, а люди младше 50 лет идут с отрицательными значениями
</div>

КОММЕНТАРИЙ ОТ УЧЕНИКА: поделила весь стаж пенсионеров на 24 часа

<div class="alert alert-danger">
<h2> Комментарии от ревьюера №1</h2>
    
Есть не явные пропуски в колонке `dob_years`
</div>

КОММЕНТАРИЙ ОТ УЧЕНИКА: Да, увидела пропуски, скорре всего они связаны с ошибкой при внесении данных, заменила их на среднее значение

<div class="alert alert-warning">
<h2> Комментарии от ревьюера №2</h2>
    
А почему кода нет с этим исправлением? Или я может что-то пропускаю
    
В рамках заполнения этих пустых значений, попробуем сделать заполнение с использованием медиан по группам пенсионер / не пенсионер
</div>



### Замена типа данных

In [5]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Метод to_numeric не подходит, потому что остаются данные типа float, поэтому для изменения типа данных был взят метод astype

### Обработка дубликатов

In [29]:
#print(data['total_income'].value_counts())
#print(data['education'].value_counts()) #дубликаты по регистру
#print(data['family_status'].value_counts()) #нет
#print(data['gender'].value_counts()) # есть одна ошибка? удалить
#print(data['purpose'].value_counts()) # нет единой логики записи, вычленить по словам

data['education'] = data['education'].str.lower()
data['children'] = data['children'].replace(-1,1)
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20,children_median)
#print(data['children'].value_counts())

#data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()


0

<div class="alert alert-danger">
<h2> Комментарии от ревьюера №1</h2>
    
Обрати внимание, что в датасете есть полностью одинаковые строки
</div>

### Вывод

В столбце education записи с разным регистром, с помощью функции str.lewer() все было приравнено к нижнему ргеистру
В столбце children были значения с -1 и 20, -1 было заменено на 1, 20 на значение медианы, т.к. тут явная ошибка и корректное значение предсказать трудно.

<div class="alert alert-success">
<h2> Комментарии от ревьюера №1</h2>
    
Тут все верно  👍
</div>

### Лемматизация

In [13]:
unique_purposes = data['purpose'].value_counts().index.tolist()
#print(data['purpose'].value_counts().to_frame())

from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

m = Mystem()
list_of_lemmas = []
for element in data['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

final_list_of_purposes = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 4 if k != ' ' if k != '\n'][0:8]
print(final_list_of_purposes)

dict_stemmed_purposes = {v:k for k,v in enumerate(final_list_of_purposes)}
dict_stemmed_purposes['недвижим'] = 0
print(dict_stemmed_purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']
{'недвижимость': 0, 'покупка': 1, 'жилье': 2, 'автомобиль': 3, 'образование': 4, 'операция': 5, 'свадьба': 6, 'строительство': 7, 'недвижим': 0}


<div class="alert alert-warning">
<h2> Комментарии от ревьюера №1</h2>
    
Не совсем понял зачем используешь Stemmer ?
    
У тебя же и так лемматизация проведена, зачем убирать окончания?
</div>

 Комментарий от ученика: окончания убираются для того чтобы если они будут разные все равно вычленить нужные группы. Например: "проведение свадьбы; кредит на садьбу" тут разные окончания, но они все равно отнесутся в одну группу

<div class="alert alert-warning">
<h2> Комментарии от ревьюера №2</h2>
    
Если ты лемматизируешь строку со словами: "проведение свадьбы кредит на свадьбу", то на выходе получишь ['проведение','свадьба','кредит','на','свадьба'], т.е. все равно пока не вижу целесообразности
</div>

print(data['purpose'].value_counts())

### Категоризация данных

In [18]:
data['purpose_cat'] = 0
for row in range(len(data)):
    for purpose in dict_stemmed_purposes:
        if purpose in data.iloc[row, 11]:
            data.iloc[row, 12] = dict_stemmed_purposes[purpose]
            
def dob_years_group (dob_years):
    if dob_years <= 22:
        return 'молодежь'
    if dob_years <= 60:
        return 'Взрослые'
    return 'пенсионеры'
data['dob_years_group'] = data['dob_years'].apply(dob_years_group)

def children_group (children):
    if children == 0:
        return 'нет детей'
    if children <= 2:
        return 'до двух детей'
    return 'многодетная семья'
data['children_group'] = data['children'].apply(children_group)

def days_employed_group (days_employed ):
    if days_employed  <= 2555:
        return 'стаж до 7 лет'
    if days_employed  <= 10950:
        return ' стаж от 7 до 30 лет'
    return 'стаж более 30 лет'
data['days_employed_group'] = data['days_employed'].apply(days_employed_group)

def total_income_group (total_income):
    if total_income <= 40000:
        return 'маленький доход'
    if total_income <=120000:
        return 'средний доход'
    if total_income <= 200000:
        return 'выше среднего'
    return 'высокий доход'
data['total_income_group'] = data['total_income'].apply(total_income_group)
                   
data.head(10)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,dob_years_group,children_group,days_employed_group,total_income_group
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,1,Взрослые,до двух детей,стаж от 7 до 30 лет,высокий доход
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0,Взрослые,до двух детей,стаж от 7 до 30 лет,средний доход
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1,Взрослые,нет детей,стаж от 7 до 30 лет,выше среднего
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4,Взрослые,многодетная семья,стаж от 7 до 30 лет,высокий доход
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0,Взрослые,нет детей,стаж более 30 лет,выше среднего
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,1,Взрослые,нет детей,стаж до 7 лет,высокий доход
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,2,Взрослые,нет детей,стаж от 7 до 30 лет,высокий доход
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,4,Взрослые,нет детей,стаж до 7 лет,выше среднего
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,0,Взрослые,до двух детей,стаж от 7 до 30 лет,средний доход
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,1,Взрослые,нет детей,стаж до 7 лет,выше среднего


### Вывод

Категоризация была исходя из целей кредитов, наличие детей в семье, трудовому стажу в годах, возврасту клиентов и их уровню дохода

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [41]:

def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

relation('children_group')


,debt
children_group,
нет детей,0.075598
многодетная семья,0.081579
до двух детей,0.092515


### Вывод

многодетные семьи имеют болше просрочек по кредиту, чем семьи без детей или с 1-2 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
relation('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


### Вывод

не замужний / не женатые чаще не возвращают кредиты

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
relation('total_income_group')

,debt
total_income_group,
высокий доход,0.070667
маленький доход,0.080000
средний доход,0.080506
выше среднего,0.087184


### Вывод

люди с доходом выше среднего чаще не возвращают кредиты

- Как разные цели кредита влияют на его возврат в срок?

In [49]:
relation('purpose_cat')

,debt
purpose_cat,
0,0.066351
2,0.068192
1,0.070203
7,0.076677
5,0.078725
6,0.080034
4,0.092200
3,0.093590


### Вывод

Люди берущие кредит на покупку автомобиля, образование или свадьбу чаще всего его не возвращют, наиболее надеждные по целям это недвижимость и жилье

### Шаг 4. Общий вывод

Наиболее не благонадёжные группы, это многодетные семьи и одинокий люди, чаще всего просрочек по кредитам нет если его берут на приобретение недвижимости

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [X]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

 <div class="alert alert-success">
<h2> Комментарии от ревьюера №1</h2>
    
Ты большая молодец, твой проект выполнен сейчас на 90% и я не могу не отправить тебе его обратно, чтобы мы все таки пришли к 100%, думаю это не займет у тебя много времени.
    
Обрати внимание на следующее:

    1) Предобработка данных (day_employed , dob_years)
    2) Посмотри про дубликаты комментарий
    3) Расскажи логику на счет Stemmer
    
Жду скорее твою работу на повторную проверку!

</div>

<div class="alert alert-success">
<h2> Комментарии от ревьюера №2</h2>
    
Обработай мои комментарии под номером 2)
</div>